In [2]:
!uv init

error: Project is already initialized in `/Users/asaucedo/Programming/real-time-serving` (`pyproject.toml` file exists)


In [3]:
!uv add mlserver

Resolved 205 packages in 0.77ms
Audited 200 packages in 0.27ms


0

In [7]:
# Set up for async background linux process runs
import os
get_ipython().system = os.system

In [8]:
!mlserver start . &

0

2024-11-09 13:59:13,594 [mlserver.parallel] DEBUG - Starting response processing loop...
2024-11-09 13:59:13,594 [mlserver.rest] INFO - HTTP server running on http://0.0.0.0:8080
2024-11-09 13:59:13,609 [mlserver.metrics] INFO - Metrics server running on http://0.0.0.0:8082
2024-11-09 13:59:13,609 [mlserver.metrics] INFO - Prometheus scraping endpoint can be accessed on http://0.0.0.0:8082/metrics
2024-11-09 13:59:13,610 [mlserver.grpc] INFO - gRPC server running on http://0.0.0.0:8081


INFO:     Started server process [78748]
INFO:     Waiting for application startup.
INFO:     Started server process [78748]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8082 (Press CTRL+C to quit)


# Server API

In [23]:
# OpenAPI Swagger Docs
!open http://localhost:8080/v2/docs/

0

INFO:     127.0.0.1:59234 - "GET /v2/docs/dataplane.json HTTP/1.1" 200 OK


In [26]:
## Info on the server
requests.get("http://localhost:8080/v2/").json()

INFO:     127.0.0.1:59330 - "GET /v2/ HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:59330 - "GET /v2 HTTP/1.1" 200 OK


{'name': 'mlserver', 'version': '1.6.1', 'extensions': []}

In [50]:
## Health of server: readiness probe on model (liveness also available on version specific)
requests.get("http://localhost:8080/v2/health/ready")

INFO:     127.0.0.1:59489 - "GET /v2/health/ready HTTP/1.1" 200 OK


<Response [200]>

## Default Reusable Runtimes
### Sklearn

In [12]:
!uv add mlserver-sklearn

Resolved 169 packages in 626ms                                       
Prepared 1 package in 257ms                                              
Installed 1 package in 6ms.6.1                              
 + mlserver-sklearn==1.6.1


In [2]:
!mkdir -p sklearn/

#### Training

In [1]:
# Original source code and more details can be found in:
# https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

# The digits dataset
digits = datasets.load_digits()

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# We learn the digits on the first half of the digits
classifier.fit(X_train, y_train)

SVC(gamma=0.001)

#### Saving the model

In [3]:
import joblib

model_file_name = "sklearn/mnist-svm.joblib"
joblib.dump(classifier, model_file_name)

['sklearn/mnist-svm.joblib']

In [64]:
!ls sklearn

mnist-svm.joblib


0

#### Serving with out of the box server

In [65]:
%%writefile sklearn/model-settings.json
{
    "name": "mnist-svm",
    "implementation": "mlserver_sklearn.SKLearnModel",
    "parameters": {
        "uri": "./mnist-svm.joblib",
        "version": "v0.1.0"
    }
}


Writing sklearn/model-settings.json


In [66]:
!ls sklearn/

mnist-svm.joblib
model-settings.json


0

#### We can now load the model using the model repository API

In [ ]:
requests.post("http://localhost:8080/v2/repository/models/mnist-svm/load")

In [67]:
import requests

x_0 = X_test[0:1]
inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": x_0.shape,
          "datatype": "FP32",
          "data": x_0.tolist()
        }
    ]
}

endpoint = "http://localhost:8080/v2/models/mnist-svm/versions/v0.1.0/infer"
response = requests.post(endpoint, json=inference_request)

response.json()

INFO:     127.0.0.1:59601 - "POST /v2/models/mnist-svm/versions/v0.1.0/infer HTTP/1.1" 404 Not Found


{'error': 'Model mnist-svm with version v0.1.0 not found'}

## MLFlow Example



In [29]:
!uv add mlserver-mlflow

Resolved 205 packages in 1.72s
Prepared 35 packages in 9.16s
Installed 35 packages in 355ms
 + alembic==1.14.0
 + blinker==1.9.0
 + cachetools==5.5.0
 + cloudpickle==3.1.0
 + contourpy==1.3.0
 + cycler==0.12.1
 + databricks-sdk==0.36.0
 + docker==7.1.0
 + flask==3.0.3
 + fonttools==4.54.1
 + gitdb==4.0.11
 + gitpython==3.1.43
 + google-auth==2.36.0
 + graphene==3.4.1
 + graphql-core==3.2.5
 + graphql-relay==3.2.0
 + gunicorn==23.0.0
 + itsdangerous==2.2.0
 + kiwisolver==1.4.7
 + mako==1.3.6
 + markdown==3.7
 + matplotlib==3.9.2
 + mlflow==2.17.2
 + mlflow-skinny==2.17.2
 + mlserver-mlflow==1.6.1
 + pillow==11.0.0
 + pyarrow==17.0.0
 + pyasn1==0.6.1
 + pyasn1-modules==0.4.1
 + pyparsing==3.2.0
 + rsa==4.9
 + smmap==5.0.1
 + sqlalchemy==2.0.36
 + sqlparse==0.5.1
 + werkzeug==3.1.3


0

In [33]:
!mkdir -p mlflow/src

0

Training

In [5]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [34]:
%%writetemplate mlflow/src/train.py
# Original source code and more details can be found in:
# https://www.mlflow.org/docs/latest/tutorials-and-examples/tutorial.html

# The data set used in this example is from
# http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties.
# In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml"
        "/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, "
            "check your internet connection. Error: %s",
            e,
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.5
    l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        model_signature = infer_signature(train_x, train_y)

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry,
            # which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(
                lr,
                "model",
                registered_model_name="ElasticnetWineModel",
                signature=model_signature,
            )
        else:
            mlflow.sklearn.log_model(lr, "model", signature=model_signature)


In [35]:
!(cd mlflow && python src/train.py)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


2024/11/09 13:13:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


0

In [17]:
import os

[experiment_file_path] = !(cd mlflow && ls --color=none -td ./mlruns/0/* | head -1)
model_path = os.path.join(experiment_file_path, "artifacts", "model")
print(model_path)

./mlruns/0/db95d5b76c8b44b58bb0dd7837b4cb18/artifacts/model


In [18]:
!ls $model_path

Makefile
README.ipynb
README.md
mlflow
pyproject.toml
uv.lock


0

In [33]:
%%writetemplate ./mlflow/model-settings.json
{{
    "name": "wine-classifier",
    "implementation": "mlserver_mlflow.MLflowRuntime",
    "parameters": {{
        "uri": "{model_path}",
        "version": "v0.1.0"
    }}
}}

#### Load using the model repository API

In [39]:
requests.post("http://localhost:8080/v2/repository/models/wine-classifier/load")

0

2024-11-09 13:36:53,592 [mlserver] WARNING - Model name 'wine-classifier' is different than model's folder name 'mlflow'.
2024-11-09 13:36:53,622 [mlserver.parallel] DEBUG - Starting response processing loop...
2024-11-09 13:36:53,623 [mlserver.rest] INFO - HTTP server running on http://0.0.0.0:8080
2024-11-09 13:36:53,635 [mlserver.metrics] INFO - Metrics server running on http://0.0.0.0:8082
2024-11-09 13:36:53,635 [mlserver.metrics] INFO - Prometheus scraping endpoint can be accessed on http://0.0.0.0:8082/metrics


INFO:     Started server process [76459]
INFO:     Waiting for application startup.
INFO:     Started server process [76459]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8082 (Press CTRL+C to quit)


2024-11-09 13:36:54,063 [mlserver.grpc] INFO - gRPC server running on http://0.0.0.0:8081
2024-11-09 13:36:55,455 [mlserver][wine-classifier:v0.1.0] INFO - Loaded model 'wine-classifier' successfully.
2024-11-09 13:36:55,457 [mlserver][wine-classifier:v0.1.0] INFO - Loaded model 'wine-classifier' successfully.


In [40]:
import requests

inference_request = {
    "inputs": [
        {
          "name": "fixed acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [7.4],
        },
        {
          "name": "volatile acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.7000],
        },
        {
          "name": "citric acid",
          "shape": [1],
          "datatype": "FP32",
          "data": [0],
        },
        {
          "name": "residual sugar",
          "shape": [1],
          "datatype": "FP32",
          "data": [1.9],
        },
        {
          "name": "chlorides",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.076],
        },
        {
          "name": "free sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [11],
        },
        {
          "name": "total sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [34],
        },
        {
          "name": "density",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.9978],
        },
        {
          "name": "pH",
          "shape": [1],
          "datatype": "FP32",
          "data": [3.51],
        },
        {
          "name": "sulphates",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.56],
        },
        {
          "name": "alcohol",
          "shape": [1],
          "datatype": "FP32",
          "data": [9.4],
        },
    ]
}

endpoint = "http://localhost:8080/v2/models/wine-classifier/infer"
response = requests.post(endpoint, json=inference_request)

response.json()

INFO:     127.0.0.1:59417 - "POST /v2/models/wine-classifier/infer HTTP/1.1" 200 OK


{'model_name': 'wine-classifier',
 'model_version': 'v0.1.0',
 'id': '6b040836-d4a9-4609-a7b8-20d7a22ceccf',
 'parameters': {'content_type': 'np'},
 'outputs': [{'name': 'output-1',
   'shape': [1, 1],
   'datatype': 'FP64',
   'parameters': {'content_type': 'np'},
   'data': [5.576883936610762]}]}

#### View the specification of the model API

In [ ]:
# Access documentation for specific model
!open http://localhost:8080/v2/models/wine-classifier/docs

In [49]:
## Health of server
requests.get("http://localhost:8080/v2/models/wine-classifier/health/live")

INFO:     127.0.0.1:59476 - "GET /v2/models/wine-classifier/health/live HTTP/1.1" 404 Not Found


<Response [404]>

In [41]:
## Info on the model inputs/outputs expected
requests.get("http://localhost:8080/v2/models/wine-classifier").json()

INFO:     127.0.0.1:59419 - "GET /v2/models/wine-classifier HTTP/1.1" 200 OK


{'name': 'wine-classifier',
 'versions': [],
 'platform': '',
 'inputs': [{'name': 'fixed acidity',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'volatile acidity',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'citric acid',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'residual sugar',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'chlorides',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'free sulfur dioxide',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'total sulfur dioxide',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'density',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 

In [45]:
## Supports multiple models, with multiple versions
requests.get("http://localhost:8080/v2/models/wine-classifier/versions/v0.1.0").json()

INFO:     127.0.0.1:59437 - "GET /v2/models/wine-classifier/versions/v0.1.0 HTTP/1.1" 200 OK


{'name': 'wine-classifier',
 'versions': [],
 'platform': '',
 'inputs': [{'name': 'fixed acidity',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'volatile acidity',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'citric acid',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'residual sugar',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'chlorides',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'free sulfur dioxide',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'total sulfur dioxide',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 'density',
   'datatype': 'FP64',
   'shape': [-1, 1],
   'parameters': {'content_type': 'np'}},
  {'name': 

# Multi-model Repository API

In [56]:
import requests

requests.post("http://localhost:8080/v2/repository/index", json={}).json()

2024-11-09 13:42:35,412 [mlserver] WARNING - Model name 'wine-classifier' is different than model's folder name 'mlflow'.
INFO:     127.0.0.1:59534 - "POST /v2/repository/index HTTP/1.1" 200 OK


[{'name': 'wine-classifier',
  'version': 'v0.1.0',
  'state': 'READY',
  'reason': ''}]

In [57]:
requests.post("http://localhost:8080/v2/repository/models/wine-classifier/unload")

2024-11-09 13:43:18,787 [mlserver][wine-classifier:v0.1.0] INFO - Unloaded version v0.1.0 of model 'wine-classifier' successfully.
2024-11-09 13:43:18,789 [mlserver][wine-classifier:v0.1.0] INFO - Unloaded version v0.1.0 of model 'wine-classifier' successfully.
2024-11-09 13:43:18,789 [mlserver] INFO - Unloaded all versions of model 'wine-classifier' successfully.
INFO:     127.0.0.1:59572 - "POST /v2/repository/models/wine-classifier/unload HTTP/1.1" 200 OK


<Response [200]>

In [58]:
requests.post("http://localhost:8080/v2/repository/index", json={}).json()

2024-11-09 13:43:29,011 [mlserver] WARNING - Model name 'wine-classifier' is different than model's folder name 'mlflow'.
INFO:     127.0.0.1:59575 - "POST /v2/repository/index HTTP/1.1" 200 OK


[{'name': 'wine-classifier',
  'version': 'v0.1.0',
  'state': 'UNAVAILABLE',
  'reason': ''}]

In [59]:
requests.post("http://localhost:8080/v2/repository/models/wine-classifier/load")

2024-11-09 13:43:39,171 [mlserver] WARNING - Model name 'wine-classifier' is different than model's folder name 'mlflow'.
2024-11-09 13:43:39,221 [mlserver][wine-classifier:v0.1.0] INFO - Loaded model 'wine-classifier' successfully.
2024-11-09 13:43:39,222 [mlserver][wine-classifier:v0.1.0] INFO - Loaded model 'wine-classifier' successfully.
INFO:     127.0.0.1:59578 - "POST /v2/repository/models/wine-classifier/load HTTP/1.1" 200 OK


<Response [200]>

In [60]:
requests.post("http://localhost:8080/v2/repository/index", json={}).json()

2024-11-09 13:43:43,352 [mlserver] WARNING - Model name 'wine-classifier' is different than model's folder name 'mlflow'.
INFO:     127.0.0.1:59580 - "POST /v2/repository/index HTTP/1.1" 200 OK


[{'name': 'wine-classifier',
  'version': 'v0.1.0',
  'state': 'READY',
  'reason': ''}]

# Building a custom runtime

In [4]:
from mlserver_sklearn import sklearn
sklearn_file = sklearn.__file__

In [5]:
!bat {sklearn_file}

───────┬────────────────────────────────────────────────────────────────────────
       │ File: /Users/asaucedo/Programming/real-time-serving/.venv/lib/python3.12/site-packages/mlserver_sklearn/sklearn.py
───────┼────────────────────────────────────────────────────────────────────────
   1   │ import joblib
   2   │ 
   3   │ from typing import List
   4   │ from sklearn.pipeline import Pipeline
   5   │ 
   6   │ from mlserver.codecs import NumpyCodec, NumpyRequestCodec, PandasCodec
   7   │ from mlserver.errors import InferenceError
   8   │ from mlserver.model import MLModel
   9   │ from mlserver.types import (
  10   │     InferenceRequest,
  11   │     InferenceResponse,
  12   │     ResponseOutput,
  13   │     RequestOutput,
  14   │ )
  15   │ from mlserver.utils import get_model_uri
  16   │ 
  17   │ PREDICT_FN_KEY = "predict_fn"
  18   │ PREDICT_OUTPUT = "predict"
  19   │ PREDICT_PROBA_OUTPUT = "predict_proba"
  20   │ PREDICT_TRANSFORM = "transform"
  21   │ VALID_OUTPUTS

In [9]:
!uv add cookiecutter

Resolved 205 packages in 0.36ms
Audited 200 packages in 0.13ms


0

In [3]:
# Run command on terminal
# > !mlserver init

#  [1/6] project_name (Lambda runtime): SimpleModel
#  [2/6] repo_user (MyGithubUsername): axsaucedo
#  [3/6] repo_name (simplemodel):
#  [4/6] class_name (Simplemodel): SimpleModel
#  [5/6] description (A short description of the project.):
#  [6/6] Select open_source_license
#    1 - MIT
#    2 - BSD-3-Clause
#    3 - No license file
#    Choose from [1/2/3] (1): 3

'/Users/asaucedo/Programming/real-time-serving/.venv/lib/python3.12/site-packages/mlserver/__init__.py'

In [4]:
!ls simplemodel/

Dockerfile           docs                 simplemodel
LICENSE              pyproject.toml       tests
Makefile             requirements-dev.txt
README.md            setup.py


In [5]:
!ls simplemodel/simplemodel/

__init__.py common.py   runtime.py  version.py


In [6]:
!bat simplemodel/simplemodel/runtime.py

───────┬────────────────────────────────────────────────────────────────────────
       │ File: simplemodel/simplemodel/runtime.py
───────┼────────────────────────────────────────────────────────────────────────
   1   │ import numpy as np
   2   │ from mlserver.model import MLModel
   3   │ from mlserver.settings import ModelSettings
   4   │ from fastapi import status
   5   │ from mlserver.utils import get_model_uri
   6   │ from mlserver.errors import InvalidModelURI, MLServerError
   7   │ from mlserver.types import (
   8   │     InferenceRequest,
   9   │     InferenceResponse,
  10   │ )
  11   │ from mlserver.codecs import NumpyCodec, NumpyRequestCodec
  12   │ from simplemodel.common import SimpleModelSettings
  13   │ 
  14   │ 
  15   │ class SimpleModel(MLModel):
  16   │     """Runtime class for specific Huggingface models"""
  17   │ 
  18   │     def __init__(self, settings: ModelSettings):
  19   │ 
  20   │         self._extra_settings = SimpleModelSettings(**settings

# Cleanup

In [6]:
!rm -rf mlflow sklearn simplemodel

0